### **Range Control Tests for the Arms** ###

**Imports and Controllers**

In [ ]:
#Raya Imports
from raya.standalone_handler import StandaloneHandler
from raya.enumerations import ANG_UNIT, POS_UNIT

#Other Imports
import matplotlib.pyplot as plt
import asyncio

In [ ]:
# Start handler
raya = StandaloneHandler()
await raya.start()

In [ ]:
# Enable controllers
arms = await raya.enable_controller('arms')

**Helpers**

In [ ]:
async def check_pick(arm, arm_joints, arm_joints_start_pose, pick, post_pick):
        '''
            INPUTS:
                pick - The pose to execute for picking the item ; dict
                post_pick - The pose to execute for lifting the item ; dict
            
            OUTPUTS:
                The function returns a boolean - True or False for valid or
                                                 invalid pick sequence
        '''
        try:
            res= await arms.is_pose_valid(arm= arm,
                                            **pick,
                                            name_trajectory ='pick_hive',
                                            save_trajectory = True,
                                            start_joints = arm_joints_start_pose,
                                            name_start_joints = arm_joints,
                                            use_start_joints = True,
                                            wait= True
                                          )
            
            joints = arms.convert_angle_joints_to_degrees(
                arm = arm,
                name_joints = arm_joints,
                angle_joints = list(res[2])
            )
            
            res = await arms.is_pose_valid(arm= arm,
                                            **post_pick, 
                                            cartesian_path = True,
                                            name_trajectory = 'post_pick_hive',
                                            save_trajectory = True,
                                            start_joints = joints,
                                            name_start_joints = arm_joints,
                                            use_start_joints = True,
                                            wait = True
                                           )
            if res[1]<0.9:
                return False
            
        except Exception as e:
            print(e)
            return False
        
        return True


async def plot_range(arm, arm_joints, arm_joints_start_pose, picks, post_pick):
     fig, axis = plt.subplots(3,3)
     for pick in picks:
        is_pick_valid = await check_pick(arm, arm_joints, arm_joints_start_pose, pick, post_pick)
        
